In [1]:
import pandas as pd

### Loading Population Data 

This data is taken from the US census from 2010 which includes up to 2019 estimates by state and region. 
The CSV is available on the US census site [here](https://www.census.gov/data/datasets/time-series/demo/popest/2010s-state-total.html#par_textimage_1873399417).

In [2]:
pop_df = pd.read_csv('nst-est2019-alldata.csv')
print("rows: {0}".format(len(pop_df.index)))
pop_df.head()

rows: 57


SUMLEV REGION DIVISION  STATE              NAME  CENSUS2010POP  \
0      10      0        0      0     United States      308745538   
1      20      1        0      0  Northeast Region       55317240   
2      20      2        0      0    Midwest Region       66927001   
3      20      3        0      0      South Region      114555744   
4      20      4        0      0       West Region       71945553   

   ESTIMATESBASE2010  POPESTIMATE2010  POPESTIMATE2011  POPESTIMATE2012  ...  \
0          308758105        309321666        311556874        313830990  ...   
1           55318443         55380134         55604223         55775216  ...   
2           66929725         66974416         67157800         67336743  ...   
3          114563030        114866680        116006522        117241208  ...   
4           71946907         72100436         72788329         73477823  ...   

   RDOMESTICMIG2019  RNETMIG2011  RNETMIG2012  RNETMIG2013  RNETMIG2014  \
0          0.000000     2.493773     2.682083     2.636187     2.921500   
1         -5.254530     0.887909    -0.038355    -0.469783    -0.986097   
2         -2.365881    -0.963930    -0.973943    -0.006924    -0.762969   
3          3.261349     5.130513     5.850458     5.292073     6.161501   
4          0.614245     2.723344     3.062896     3.162262     4.026429   

   RNETMIG2015  RNETMIG2016  RNETMIG2017  RNETMIG2018  RNETMIG2019  
0     3.260435     3.252788     2.871957     2.153911     1.818059  
1    -2.061965    -2.490484    -1.837048    -2.134447    -2.859713  
2    -1.388437    -1.241784    -0.557370    -0.922755    -1.111173  
3     7.277358     7.150074     6.198168     5.225519     5.203720  
4     4.987285     5.261078     4.021194     3.044951     2.312083  

[5 rows x 151 columns]

### Loading GDP and Vote Data

This data is taken from the US Dept. of Commerce GDP measures per state for 2019 Q1-Q4 and 2020 Q1-Q2. The original Excel sheet contains additional tables and can be found on the BEA website [here](https://www.bea.gov/news/2020/gross-domestic-product-state-2nd-quarter-2020). This analysis is only performed on 2019 data and election results are taken from the Wall Street Journal's Live Election Results found [here](https://www.wsj.com/election-results-2020/). **NOTE: AS OF 11/6/2020 THESE ARE ESTIMATES OF STATE RESULTS AND ARE NOT FINAL**.

In [3]:
# Get GDP for each quarter in 2019 per state in addition to vote (Biden v. Trump)
# State name field labelled 'NAME' to match census CSV above
gdp_df = pd.read_excel('2019_gdp_and_vote_by_state.xlsx')

# Excel sheet contains some strange spacing for the name, removing it to match census data
gdp_df['NAME'] = gdp_df['NAME'].apply(lambda x: x.strip())

print("rows: {0}".format(len(gdp_df.index)))
gdp_df.head()

rows: 51


NAME  2019-q1  2019-q2  2019-q3  2019-q4  vote
0  California  3063191  3119174  3143837  3205000     1
1       Texas  1827426  1835576  1850626  1861582     0
2    New York  1746191  1771545  1779740  1791567     1
3     Florida  1087641  1098679  1113170  1126510     0
4    Illinois   878173   880445   890358   893356     1

### Connect Population, GDP, and Vote Data

Link the US 2019 population estimate with the quarterly GDP and 2020 election vote for each state.

In [4]:
combined_df = pop_df[['NAME','POPESTIMATE2019']].merge(gdp_df,how='inner',on='NAME')
print("rows: {0}".format(len(combined_df.index)))
combined_df.head()

rows: 51


NAME  POPESTIMATE2019  2019-q1  2019-q2  2019-q3  2019-q4  vote
0     Alabama          4903185   225565   226927   229328   230750     0
1      Alaska           731545    54138    54280    54450    54675     0
2     Arizona          7278717   361380   366905   373173   379019     1
3    Arkansas          3017804   129193   130471   131556   132596     0
4  California         39512223  3063191  3119174  3143837  3205000     1

### Compute State GDP, GDP per capita

In this cell, compute the total GDP for 2019 for each state as well as the GDP per capita and add these columns to the data set. Note that the BEA data table provides quarterly GDP in millions of dollars. 

In [5]:
combined_df['2019-gdp_mil'] = combined_df[['2019-q1','2019-q2','2019-q3','2019-q4']].sum(axis=1)

# Multiply by 1,000,000 to get actual GDP per capita 
combined_df['2019-gdp_per_capita'] = (combined_df['2019-gdp_mil']*1000000)/combined_df['POPESTIMATE2019']
combined_df.head()

NAME  POPESTIMATE2019  2019-q1  2019-q2  2019-q3  2019-q4  vote  \
0     Alabama          4903185   225565   226927   229328   230750     0   
1      Alaska           731545    54138    54280    54450    54675     0   
2     Arizona          7278717   361380   366905   373173   379019     1   
3    Arkansas          3017804   129193   130471   131556   132596     0   
4  California         39512223  3063191  3119174  3143837  3205000     1   

   2019-gdp_mil  2019-gdp_per_capita  
0        912570        186117.798941  
1        217543        297374.734295  
2       1480477        203398.071391  
3        523816        173575.222248  
4      12531202        317147.481173

### Perform the Analysis

In this cell, various analysis is performed on the data set of the 50 states and Washington D.C.. Remember, a vote of 1 represents a state vote for Biden and a vote of 0 represents a state vote for Trump.

#### By Vote GDP

Compute the total GDP of states that voted for Biden and the total GDP of the states that voted for Trump. Note that Maine and Nebraska are special cases due to their electoral college systems. In Maine and Nebraska, the candidate who receives the 2 state wide votes are the state that they are counted under. The congressional votes are ignored. 

#### By Vote GDP per Capita 

Compute the total GDP per capita of states that voted for Biden and the total GDP per capita of the states that voted for Trump. 

#### Top and Bottom 10 states by GDP 

Sort states on their total 2019 GDP and store the top and bottom 10. Lastly, for both the top and bottom 10 count the number of states that voted for Biden and Trump. 

#### Top and Bottom 10 states by GDP per capita

Sort states on their total 2019 GDP per capita and store the top and bottom 10. Lastly, for both the top and bottom 10 count the number of states that voted for Biden and Trump. 

In [6]:
# Sum the GDP in millions for each candidate 
by_vote_gdp = combined_df.groupby(['vote'])['2019-gdp_mil'].sum()

# Sump the GDP per capita (USD) for each candidate 
by_vote_gdp_per_capita = combined_df.groupby(['vote'])['2019-gdp_per_capita'].sum()

# Sort states based on 2019 GDP in descending order; reset and drop the old index so top to bottom will be 0...50
gdp_sort = combined_df.sort_values(by=['2019-gdp_mil'],ascending=False)[['NAME','vote']].reset_index(drop=True)

# Adjust so index is 1...51 
gdp_sort.index += 1 

# Get top and bottom 10 
top_10_gdp = gdp_sort.head(10)
btm_10_gdp = gdp_sort.tail(10)

# Get candidate counts for top and bottom 10 
top_gdp_counts = top_10_gdp.groupby(['vote'])['vote'].count()
btm_gdp_counts = btm_10_gdp.groupby(['vote'])['vote'].count()

# Sort states based on 2019 GDP per capita in descending order; reset and drop the old index so top to bottom will be 0...50
gdp_per_capita_sort = combined_df.sort_values(by=['2019-gdp_per_capita'],ascending=False)[['NAME','vote']].reset_index(drop=True)

# Adjust so index is 1...51 
gdp_per_capita_sort.index += 1 

# Get top and bottom 10 
top_10_gdp_per_capita = gdp_per_capita_sort.head(10)
btm_10_gdp_per_capita = gdp_per_capita_sort.tail(10)

# Get candidate counts for top and bottom 10 
top_gdp_per_capita_counts = gdp_per_capita_sort.head(10).groupby(['vote'])['vote'].count()
btm_gdp_per_capita_counts = gdp_per_capita_sort.tail(10).groupby(['vote'])['vote'].count()

#### Display Report 

In this cell, a report is printed of the analysis performed in the previous cell.

In [7]:
print("----- TOTAL GDP (Millions USD) ------\n")
print("Biden: %d" % (by_vote_gdp[1]))
print("Trump: %d\n" % (by_vote_gdp[0]))
print("Biden/Trump ratio: %.2f\n" % (by_vote_gdp[1]/by_vote_gdp[0]))
print("----- TOTAL GDP per capita (USD) ------\n")
print("Biden: %d" % (by_vote_gdp_per_capita[1]))
print("Trump: %d\n" % (by_vote_gdp_per_capita[0]))
print("Biden/Trump ratio: %.2f\n" % (by_vote_gdp_per_capita[1]/by_vote_gdp_per_capita[0]))
print("----- TOP 10 STATES (2019 GDP) ----- \n\n", top_10_gdp, "\n")
print("Number for Biden: %d" % (top_gdp_counts[1]))
print("Number for Trump: %d\n" % (top_gdp_counts[0]))
print("----- BOTTOM 10 STATES (2019 GDP) ----- \n\n", btm_10_gdp, "\n")
print("Number for Biden: %d" % (btm_gdp_counts[1]))
print("Number for Trump: %d\n" % (btm_gdp_counts[0]))
print("----- TOP 10 STATES (2019 GDP per capita) ----- \n\n", top_10_gdp_per_capita, "\n")
print("Number for Biden: %d" % (top_gdp_per_capita_counts[1]))
print("Number for Trump: %d\n" % (top_gdp_per_capita_counts[0]))
print("----- BOTTOM 10 STATES (2019 GDP per capita) ----- \n\n", btm_10_gdp_per_capita, "\n")
print("Number for Biden: %d" % (btm_gdp_per_capita_counts[1]))
print("Number for Trump: %d" % (btm_gdp_per_capita_counts[0]))

----- TOTAL GDP (Millions USD) ------

Biden: 53914246
Trump: 31339882

Biden/Trump ratio: 1.72

----- TOTAL GDP per capita (USD) ------

Biden: 7456867
Trump: 5595069

Biden/Trump ratio: 1.33

----- TOP 10 STATES (2019 GDP) ----- 

             NAME  vote
1     California     1
2          Texas     0
3       New York     1
4        Florida     0
5       Illinois     1
6   Pennsylvania     1
7           Ohio     0
8     New Jersey     1
9        Georgia     1
10    Washington     1 

Number for Biden: 7
Number for Trump: 3

----- BOTTOM 10 STATES (2019 GDP) ----- 

              NAME  vote
42  West Virginia     0
43       Delaware     1
44          Maine     1
45   Rhode Island     1
46   North Dakota     0
47   South Dakota     0
48         Alaska     0
49        Montana     0
50        Wyoming     0
51        Vermont     1 

Number for Biden: 4
Number for Trump: 6

----- TOP 10 STATES (2019 GDP per capita) ----- 

                     NAME  vote
1   District of Columbia     1
2      